In [1]:
import sqlite3 as db
import pandas as pd

In [2]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.transform import factor_cmap
from bokeh.io import output_notebook

# Initialize Bokeh to output in the notebook
output_notebook()

def make_barchart(df, x_col, y_col, kwargs_figure=None):
    if kwargs_figure is None:
        kwargs_figure = {}

    # Prepare the data source for Bokeh
    source = ColumnDataSource(df)

    # Get unique values for x-axis
    x_values = df[x_col].unique().tolist()

    # Define a suitable palette based on the number of unique x_values
    palette = ['#3182bd', '#6baed6', '#9ecae1', '#c6dbef', '#e6550d', '#fd8d3c', '#fdae6b', '#fdd0a2', '#31a354', '#74c476']

    # Create a Bokeh figure
    p = figure(x_range=x_values, **kwargs_figure)

    # Create color map based on the factors and palette
    colors = factor_cmap(x_col, palette=palette, factors=x_values)

    # Add bars to the figure
    p.vbar(x=x_col, top=y_col, width=0.9, source=source, legend_field=x_col, fill_color=colors)

    # Customize the figure
    p.xgrid.grid_line_color = None
    p.y_range.start = 0
    p.legend.orientation = "horizontal"
    p.legend.location = "top_center"

    return p

Loading BokehJS ...

In [3]:
df = pd.read_csv('/Users/dariushammond/Desktop/Projects/Bank Failures/bank-data.csv')

disk_engine = db.connect('bank_failures.db')

c = disk_engine.cursor()
df.to_sql("bankfailures", disk_engine, if_exists='replace', index=False)


3548

# Bank Failures Dataset

## Columns and Descriptions

1. **Institution Name**
   - **Description:** The name of the failed bank.
   
2. **Cert**
   - **Description:** The unique certificate number assigned to the bank by the FDIC.
   
3. **FIN**
   - **Description:** The Federal Information Number, a unique identifier for financial institutions.
   
4. **Location**
   - **Description:** The city and state where the bank was headquartered.
   
5. **Effective Date**
   - **Description:** The date when the bank failure was officially recorded.
   
6. **Insurance Fund**
   - **Description:** The insurance fund responsible for covering the bank's deposits.
   
7. **Resolution**
   - **Description:** The method used to resolve the bank's failure, such as payout or acquisition by another bank.
   
8. **Estimated Loss**
   - **Description:** The estimated financial loss resulting from the bank's failure.
   
9. **Transaction Type**
   - **Description:** The type of transaction used to resolve the bank, such as a purchase and assumption (P&A) or insured deposit transfer (IDT).
   
10. **Charter Class**
    - **Description:** The charter class of the bank, indicating whether it was a national or state-chartered bank.
    
11. **Total Deposits**
    - **Description:** The total amount of deposits held by the bank at the time of its failure.
    
12. **Total Assets**
    - **Description:** The total amount of assets held by the bank at the time of its failure.


In [4]:
df = pd.read_sql_query('SELECT * FROM BANKFAILURES;', disk_engine)
df.head()

,CERT,CHCLASS1,CITYST,COST,FAILDATE,FIN,ID,NAME,QBFASSET,QBFDEP,RESTYPE,RESTYPE1,SAVR
0,27332,NM,"PHILADELPHIA, PA",NaN,4/26/2024,10546,4110,REPUBLIC BANK,5866190.0,4373927.0,FAILURE,PA,BIF
1,8758,NM,"SAC CITY, IA",14804.0,11/3/2023,10545,4109,CITIZENS BANK,60448.0,52311.0,FAILURE,PA,DIF
2,25851,SM,"ELKHART, KS",54167.0,7/28/2023,10544,4108,HEARTLAND TRI-STATE BANK,139446.0,130110.0,FAILURE,PA,DIF
3,59017,NM,"SAN FRANCISCO, CA",16566779.0,5/1/2023,10543,4107,FIRST REPUBLIC BANK,212638872.0,176436706.0,FAILURE,PA,DIF
4,57053,NM,"NEW YORK, NY",2787270.0,3/12/2023,10540,4106,SIGNATURE BANK,110363650.0,88612911.0,FAILURE,PA,DIF


## Renaming columns
Looking at the dataframe we can see that the column names arent that clear. We can rename the column names in Table by using the **ALTER TABLE** command.

In [5]:
new_columns_names = [
            ('CHCLASS1', 'CHARTER_CLASS'),
             ('CITYST' , 'CITY_ST'),
             ('COST', 'ESTIMATED_LOSS'),
             ('FAILDATE', 'FAIL_DATE'),
             ('QBFASSET', 'TOTAL_ASSETS'),
             ('QBFDEP', 'TOTAL_DEPOSITS'),
             ('RESTYPE', 'RESOLUTION'),
             ('RESTYPE1', 'TRANSACTION_TYPE'),
             ('SAVR', 'INSURANCE_FUND')]
for old_name, new_name in new_columns_names: 
    c.execute(f'ALTER TABLE BANKFAILURES RENAME COLUMN {old_name} TO {new_name}') #executes query string

    

In [6]:
#disk_engine.commit() # commit so changes can be perma

# How many banks have failed in this dataset?

In [7]:
query = '''SELECT COUNT(*) AS number_of_failed_banks
            FROM BANKFAILURES
            
'''
df = pd.read_sql_query(query,disk_engine).rename(columns=str.lower);df.head()

,number_of_failed_banks
0,3548


# What are the unique locations where banks have failed?

In [8]:
query = '''SELECT DISTINCT city_st
            FROM BANKFAILURES'''
df = pd.read_sql_query(query,disk_engine).rename(columns=str.lower)
print("There are {} unique locations, the first five are:".format(len(df)))
df.head()

There are 1998 unique locations, the first five are:


,city_st
0,"PHILADELPHIA, PA"
1,"SAC CITY, IA"
2,"ELKHART, KS"
3,"SAN FRANCISCO, CA"
4,"NEW YORK, NY"


## Lets see how many by state, whats your guess? 50?

In [9]:
query = ''' SELECT DISTINCT SUBSTR(city_st,-2) AS State, COUNT(*) AS Count
            FROM BANKFAILURES
            GROUP BY 1
            ORDER BY Count DESC         
            '''
df = pd.read_sql_query(query,disk_engine)
print("There are {} unique States, the first five are:".format(len(df)))
df.head()

There are 53 unique States, the first five are:


,State,Count
0,TX,863
1,CA,259
2,IL,200
3,FL,194
4,OK,169


## We also see the counts for each state and looks like texas has a lot of banks that failed. Lets calculate a Cumulative percentage

In [10]:
query = '''
        SELECT 
    State, 
    Count, 
    ROUND((Count * 1.0/ (SELECT COUNT(*) FROM BANKFAILURES)) * 100.00000,2) || '%' AS TotalFailuresPercentageByState
FROM (
    SELECT 
        SUBSTR(city_st, -2) AS State, 
        COUNT(*) AS Count
    FROM BANKFAILURES
    GROUP BY SUBSTR(city_st, -2)
)
ORDER BY Count DESC     
 
'''
df = pd.read_sql_query(query,disk_engine)
print("The Top 5 States with the highest percentage of bank failures")

df.head()

The Top 5 States with the highest percentage of bank failures


,State,Count,TotalFailuresPercentageByState
0,TX,863,24.32%
1,CA,259,7.3%
2,IL,200,5.64%
3,FL,194,5.47%
4,OK,169,4.76%


### The Top 10 States with the most Bank Failures and we can see that Texas holds almost a fourth of the total bank failures

In [11]:
p = make_barchart(df.head(10), 'State', 'Count', kwargs_figure={'width': 640, 'height': 320})
show(p)